
![ImageName](pic/title.jpg)  

**作者：[lqy](https://www.heywhale.com/home/user/profile/5f27fd9633e1be002cc65a1d)** 华东师范大学气象学研究生、和鲸社区气象数据科学频道版主

🐋：本项目来自和鲸社区《[气象训练营⑦：WRF模式后处理](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1)》活动，所有教案代码都可以一键跑通，你可以 fork 后在线在线运行、调试学习、完成作业练习。  

学习过程中如果你遇到任何问题，欢迎使用搜索引擎，或在 [讨论区](https://www.heywhale.com/home/competition/forumlist/64478fec113e81a18dc70cd1) 中发帖提出，我们很乐意为你提供帮助。

# 关卡四：计算WRF模拟的衍生变量

WRF 模拟数据里的变量非常多，可以达到上百个变量，用于描述**大气运动基本属性和关键物理过程**，能够对大气运动进行多维度的分析。  

在不同的业务分析和科研方向上，我们感兴趣的 WRF 模拟变量可能存在差异，那么 WRF 模式能否提供我们所需的变量，这一点是大家非常关心的。  

如果发现变量在WRF模拟数据中没有直接输出（不用担心），我们可以通过 WRF 基础变量计算得到 WRF 模拟的衍生变量，例如：**能见度、云顶亮温、云覆盖**等数据。  


In [ ]:
# 导入模块
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from wrf import getvar
import metpy.calc as mpcalc
from metpy.units import units
import cmaps

In [ ]:
# WRF数据目录
wrfout_path = './dataset/wrfout/'

# 用netCDF4包读取数据
wrf_file = Dataset(wrfout_path + 'wrfout_d01_2019-08-08_18_00_00')

## 一、基于wrf-python计算WRF衍生变量

getvar函数十分强大，不仅能够提取 WRF 模式的输出变量，还能直接计算 WRF 衍生变量。  

> WRF常用的诊断变量（getvar函数）：https://wrf-python.readthedocs.io/en/latest/diagnostics.html  

### （1）云顶亮温

云顶亮温是云顶的温度，通常是由气象卫星进行观测，可以表征云顶高度和大气垂直对流的发展情况。云顶亮温越低，说明云的发展高度越高，大气对流越旺盛。

In [ ]:
# 提取云顶亮温（Cloud Top Temperature）
ctt = getvar(wrf_file, 'ctt')
ctt.plot(x='XLONG', y='XLAT', figsize=(10,6))

### （2）云覆盖

云覆盖是所有的某个高度层的云占天空的百分比，数值范围为0-1，0即表示晴天，1即表示阴天/雨天。WRF模拟提供了低层、中层和高层的云覆盖。

In [ ]:
cloudfrac = getvar(wrf_file, 'cloudfrac')
cloudfrac

### ✍ 小练习：分别绘制低层、中层和高层的云覆盖

In [ ]:
### （你的代码）###
# 提示：用索引来获取低层、中层和高层云覆盖
# 低层云覆盖
cloudfrac[0]

# 中层云覆盖
cloudfrac[1]

# 高层云覆盖
cloudfrac[2]

### （3）可降水量

可降水量是指空气柱里含有的水汽的总数量。它对应于空气中的水分全部凝结成雨、雪降落所能形成的降水量，即理论上当前大气状态的降雨量极大值。

In [ ]:
# 可降水量
pw = getvar(wrf_file, 'pw')
pw.plot(x='XLONG', y='XLAT', figsize=(10,6))

### （4）雷达反射率

不同高度层的 WRF 模拟的雷达反射率，可以与地面雷达的反射率进行比对

In [ ]:
dbz = getvar(wrf_file, 'dbz')
dbz

### （5）最大雷达反射率

WRF 模拟的最大雷达反射率，局地的对流云团更为明显

In [ ]:
mdbz = getvar(wrf_file, 'mdbz')
mdbz.plot(x='XLONG', y='XLAT', figsize=(10,6))

### （6）有效对流位能

通过`cape_2d`提取二维（不包含高度层）的有效对流位能，共返回四个变量。  

+ mcape：最大有效对流位能 (Max convective available potential energy）  
+ mcin：最大对流抑制能 (Max convective inhibition）  
+ lcl：对流凝结高度（convective condensation level）  
+ lfc：自由对流高度（level of Free Convection）

In [ ]:
# 提取二维有效对流位能
cape_2d = getvar(wrf_file, 'cape_2d')
cape_2d

In [ ]:
cape_2d[0].plot(x='XLONG', y='XLAT', figsize=(10,6))

通过`cape_3d`提取三维（包含高度层）的有效对流位能，共返回两个变量。  

+ cape：有效对流位能 (convective available potential energy）  
+ cin：对流抑制能 (convective inhibition）

In [ ]:
# 提取三维有效对流位能
cape_3d = getvar(wrf_file, 'cape_3d')
cape_3d

下面我们来介绍另一种提取诊断变量的方法，特定变量导入特定函数来提取。  

该方法可以支持更多的诊断变量，具体衍生变量说明：https://wrf-python.readthedocs.io/en/latest/internal_api/index.html 。

### （7）绝对涡度

当有水平辐散时，绝对涡度减小；有水平辐合时，绝对涡度增加。

In [ ]:
from wrf.g_vorticity import get_avo

avo = get_avo(wrf_file)
avo

### （8）位势涡度

**位势涡度是一个标量，是一个综合表征大气运动状态和热力状态的物理量**，它的重要性在于绝热无摩擦运动中微团的位涡守恒，位涡守恒定理揭示了涡度变化是受到大气热力结构制约的。位势涡度是由绝对涡度和静力稳定度两部分组成的，平流层和对流层顶及高空锋区附近的高位涡主要是因为温度垂直梯度很大。

In [ ]:
from wrf.g_vorticity import get_pvo

pvo = get_pvo(wrf_file)
pvo

### （9）垂直速度

**气块在垂直方向上的运动速度**，此处单位是$Pa$  $s^{-1}$ 。**数值为正，表示下沉运动；数值为负，表示上升运动。**

In [ ]:
from wrf.g_omega import get_omega

omega = get_omega(wrf_file)
omega

## 二、Stager和Destager

仔细观察的小伙伴会发现如下这些函数，它们都带有destag标签，而且都是风速相关的变量。这究竟是怎么回事呢？

  
![Image Name](pic/destag.png)  


让我们再回来看原始的WRF模拟数据，不难发现wrfout里有些变量输出在交错格点(stag)，为了计算中保持变量维度一致，需要destag(去stag)。  



![Image Name](pic/destag_2.png)  


特别的是，上一关案例中我们给出了风速在地图投影上的绘制方法，而这里destag函数的功能是为了与观测资料进行比较。所以两者并不冲突，需要根据需求选择合适的函数。

In [ ]:
from wrf.g_wind import get_destag_wspd_wdir

get_destag_wspd_wdir(wrf_file)

### ✍ 小练习：比较原始风速和Destag后的风速大小

本次案例主要介绍了衍生变量的计算工具以及常用的衍生变量的计算方法。  通过wrf-python模块，我们可以计算非常多的WRF衍生变量，**计算这些变量是为了更好地验证模拟的结果**，例如通过卫星观测的云顶亮温、地面雷达观测反射率来评估WRF模拟结果。  

当然，如果wrf-python模块里没有我们想要的变量，也不妨根据相关公式原理进行计算。例如：[利用wrfout数据计算能见度 | 气象备忘录](https://www.heywhale.com/mw/project/6210d9b368364e0017a4659f)。  

希望大家通过本次学习，能快速地找到并计算自己想要的模拟变量。

In [ ]:
### （你的代码）###



恭喜你完成了WRF后处理训练营第四关的学习材料，能够利用wrf-python计算更多的模拟变量，对数值模式的结果进行更深入的诊断分析。

## 闯关题  

### STEP1：根据要求完成题目：  


基于`wrfout_d01_2019-08-09_00_00_00`时刻的 WRF 模拟数据进行分析，回答以下问题：  


1. 绘制云顶亮温分布，其最低值为多少℃ ？（使用 round 取整）      


2. 绘制可降水量分布，其最大值为多少mm ？（使用 round 取整）      


3. 绘制最大雷达反射率分布，其最大值为多少dBZ ？（使用 round 取整）      

In [ ]:
# 填入你的答案
answer_1 = ''
answer_2 = ''
answer_3 = '' 

## ⚡ 下一关预告：  

下一关我们将介绍 WRF 模拟数据的**不同插值场景及常用插值方法**，让模式数据分析更加丝滑~~